#### Import Libraries and set Spark Config

In [ ]:
import sempy
import sempy.fabric as fabric
import sempy_labs as labs
from pyspark.sql.functions import col, count
from datetime import datetime, timedelta

In [ ]:
spark.conf.set("spark.databricks.delta.properties.defaults.minWriterVersion", 5)
spark.conf.set("spark.databricks.delta.properties.defaults.minReaderVersion", 2)

#### Fetch Activity Log Data & Save it a Delta Table
(Comment or remove below cell when you will schedule this notebook to run daily)

In [ ]:
log_date = '2025-08-15'

df_audit_logs = labs.admin.list_activity_events(start_time=f'{log_date}T00:00:00', end_time=f'{log_date}T23:59:59')
display(df_audit_logs)

spark_df_audit_logs = spark.createDataFrame(df_audit_logs)
spark_df_audit_logs.write.mode("overwrite").format("delta").option("delta.columnMapping.mode", "name").saveAsTable("fact_audit_log_v2")


#### Get logs for last 25 days 
(Comment or remove below cell when you will schedule this notebook to run daily)

In [ ]:
# Today's date
today = datetime.today()

start_date = today - timedelta(days=25)

# Loop through the last 25 days
for i in range(25):
    date = start_date + timedelta(days=i)
    formated_date = date.strftime('%Y-%m-%d')
    print(formated_date)
    df_audit_logs = labs.admin.list_activity_events(start_time=f'{formated_date}T00:00:00', end_time=f'{formated_date}T23:59:59')

    if df_audit_logs.shape[0] == 0:
        print("nothing to write")

    else:
        # Convert to Spark DataFrame
        spark_df_audit_logs = spark.createDataFrame(df_audit_logs)
        spark_df_audit_logs.write.format("Delta").mode("append").save("your-tables-abfs-path")
        print("Data written")



#### Append Yesterday's Data (For Daily Incremental Load)

In [ ]:
yesterday = datetime.today() - timedelta(days=1)

formated_date = yesterday.strftime('%Y-%m-%d')

print(formated_date)

df_audit_logs = labs.admin.list_activity_events(start_time=f'{formated_date}T00:00:00', end_time=f'{formated_date}T23:59:59')

if df_audit_logs.shape[0] == 0:
    print("nothing to write")

else:
    # Convert to Spark DataFrame
    spark_df_audit_logs = spark.createDataFrame(df_audit_logs)
    spark_df_audit_logs.write.format("Delta").mode("append").save("your-tables-abfs-path")